In [4]:
#라이브러리 실행
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
data=pd.read_excel('아이템기반최종데이터.xlsx')
data.head()

,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수
0,0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,미니원피스,"0, 0, 0",무지,단추,겨울,0.0,모던시크,0.0
1,1,[DAYONE LABEL]로티 숏팬츠,연청,반바지,"190, 210, 237",무지,포켓,봄 여름,0.0,심플베이직,0.0
2,2,[DAYONE LABEL]슬로 보트넥티,옐로우,티셔츠,"245, 230, 160",무지,보트넥,봄 가을,0.0,청순,0.0
3,3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,롱원피스,"220, 174, 166",데님,크롭,여름,0.0,청순,0.0
4,4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,스키니,"85, 100, 113",무지,슬림,겨울,0.0,캐주얼,0.0


In [6]:
data=data[['상품명','카테고리','패턴재질','모양','계절','스타일','종합점수']]
data.head()

,상품명,카테고리,패턴재질,모양,계절,스타일,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),미니원피스,무지,단추,겨울,모던시크,0.0
1,[DAYONE LABEL]로티 숏팬츠,반바지,무지,포켓,봄 여름,심플베이직,0.0
2,[DAYONE LABEL]슬로 보트넥티,티셔츠,무지,보트넥,봄 가을,청순,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스,롱원피스,데님,크롭,여름,청순,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),스키니,무지,슬림,겨울,캐주얼,0.0


In [7]:
data.shape

(3503, 7)

In [9]:
df=data.drop_duplicates(['상품명'])
df.shape

(1845, 7)

In [10]:
df.head()

,상품명,카테고리,패턴재질,모양,계절,스타일,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),미니원피스,무지,단추,겨울,모던시크,0.0
1,[DAYONE LABEL]로티 숏팬츠,반바지,무지,포켓,봄 여름,심플베이직,0.0
2,[DAYONE LABEL]슬로 보트넥티,티셔츠,무지,보트넥,봄 가을,청순,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스,롱원피스,데님,크롭,여름,청순,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),스키니,무지,슬림,겨울,캐주얼,0.0


In [13]:
df['종합컬럼']=df['스타일']+'-'+df['계절']+'-'+df['카테고리']+'-'+df['모양']+'-'+df['패턴재질']
df.head()

<ipython-input-13-a572d1700fdf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['종합컬럼']=df['스타일']+'-'+df['계절']+'-'+df['카테고리']+'-'+df['모양']+'-'+df['패턴재질']


,상품명,카테고리,패턴재질,모양,계절,스타일,종합점수,종합컬럼
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),미니원피스,무지,단추,겨울,모던시크,0.0,모던시크-겨울 -미니원피스-단추-무지
1,[DAYONE LABEL]로티 숏팬츠,반바지,무지,포켓,봄 여름,심플베이직,0.0,심플베이직-봄 여름 -반바지-포켓-무지
2,[DAYONE LABEL]슬로 보트넥티,티셔츠,무지,보트넥,봄 가을,청순,0.0,청순-봄 가을 -티셔츠-보트넥-무지
3,[DAYONE LABEL]워프 스퀘어롱원피스,롱원피스,데님,크롭,여름,청순,0.0,청순-여름 -롱원피스-크롭-데님
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),스키니,무지,슬림,겨울,캐주얼,0.0,캐주얼-겨울 -스키니 -슬림-무지


In [14]:
df=df[['상품명','종합컬럼','종합점수']]
df.head()

,상품명,종합컬럼,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),모던시크-겨울 -미니원피스-단추-무지,0.0
1,[DAYONE LABEL]로티 숏팬츠,심플베이직-봄 여름 -반바지-포켓-무지,0.0
2,[DAYONE LABEL]슬로 보트넥티,청순-봄 가을 -티셔츠-보트넥-무지,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스,청순-여름 -롱원피스-크롭-데님,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),캐주얼-겨울 -스키니 -슬림-무지,0.0


In [16]:
count_vector = CountVectorizer(ngram_range=(1,1)) #단어의 묶음

In [17]:
c_vector=count_vector.fit_transform(df['종합컬럼'])

In [18]:
genre_c_sim=cosine_similarity(c_vector,c_vector).argsort()[:,::-1] 

In [19]:
genre_c_sim.shape

(1845, 1845)

In [20]:
def get_recommend_item_list(df,상품이름,top=30): 
    #특정 영화와 비슷한 영화를추천해야 하기에 '특정 영화' 정보를 추출
    target_item_index=df[df['상품명']==상품이름].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_index=genre_c_sim[target_item_index,:top].reshape(-1)
  
    #본인 제외
    sim_index=sim_index[sim_index != target_item_index]

    #data frame로 만들고 vote_count로 정렬한뒤 return
    result=df.iloc[sim_index].sort_values('종합점수',ascending=False)[:10]
    
    #결과를 추출하기에 앞서 합쳤던 데이터들을 원래대로 분리시킴
    result['스타일']=result.종합컬럼.str.split('-').str[0]
    result['계절']=result.종합컬럼.str.split('-').str[1]
    result['카테고리']=result.종합컬럼.str.split('-').str[2]
    result['모양']=result.종합컬럼.str.split('-').str[3]
    result['패턴재질']=result.종합컬럼.str.split('-').str[4]
    
    # 필요 칼럼만 추가하고, 중복되서 나오게 되는 데이터는 제거
    results=result[['상품명','스타일','계절','카테고리','모양','패턴재질']]
    results=results.drop_duplicates()
    
    return results

In [21]:
get_recommend_item_list(df,'[La belle rose]론다 플리츠 미니스커트')

,상품명,스타일,계절,카테고리,모양,패턴재질
799,[La belle rose]로제 스퀘어 미니훌원피스,심플베이직,여름,미니원피스,스퀘어넥,무지
103,[La belle rose]프레인 퍼프 스퀘어티,러블리,봄 여름,티셔츠,스퀘어넥,무지
3135,세이디 퍼프 미니원피스(타이set),러블리,봄 여름,미니원피스,랩,무지
579,위퍼 스퀘어원피스,로맨틱,봄 여름 가을,미니원피스,스퀘어넥,무지
144,드밍 플라워 미니원피스,러블리,,미니원피스,스퀘어넥,플라워
283,스칼렛 튤립스커트,러블리,여름,미니스커트,랩,무지
2725,[La belle rose]히어 뷔스티에원피스&랩가디건set,청순,여름,미니원피스,랩,무지
238,미포유 랩 셔링블라우스,러블리,여름,블라우스,랩,무지
2713,[La belle rose]퍼니 입술넥원피스,러블리,봄,미니원피스,보트넥,무지
2781,그랑드 셔링 랩원피스(벨트set),페미닌,여름,미니원피스,랩,무지


# Case 2 기술조교님

In [64]:
data=pd.read_excel('아이템기반최종데이터.xlsx')
data.head()

,Unnamed: 0,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,종합점수
0,0,[DAYONE LABEL]렌프 자켓원피스(벨트set),블랙,미니원피스,"0, 0, 0",무지,단추,겨울,0.0,모던시크,0.0
1,1,[DAYONE LABEL]로티 숏팬츠,연청,반바지,"190, 210, 237",무지,포켓,봄 여름,0.0,심플베이직,0.0
2,2,[DAYONE LABEL]슬로 보트넥티,옐로우,티셔츠,"245, 230, 160",무지,보트넥,봄 가을,0.0,청순,0.0
3,3,[DAYONE LABEL]워프 스퀘어롱원피스,피치핑크,롱원피스,"220, 174, 166",데님,크롭,여름,0.0,청순,0.0
4,4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),블루,스키니,"85, 100, 113",무지,슬림,겨울,0.0,캐주얼,0.0


In [65]:
data=data[['상품명','카테고리','패턴재질','모양','계절','스타일','종합점수']]
data.head()

,상품명,카테고리,패턴재질,모양,계절,스타일,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),미니원피스,무지,단추,겨울,모던시크,0.0
1,[DAYONE LABEL]로티 숏팬츠,반바지,무지,포켓,봄 여름,심플베이직,0.0
2,[DAYONE LABEL]슬로 보트넥티,티셔츠,무지,보트넥,봄 가을,청순,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스,롱원피스,데님,크롭,여름,청순,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),스키니,무지,슬림,겨울,캐주얼,0.0


In [66]:
df=data.drop_duplicates(['상품명'])
df.head()

,상품명,카테고리,패턴재질,모양,계절,스타일,종합점수
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),미니원피스,무지,단추,겨울,모던시크,0.0
1,[DAYONE LABEL]로티 숏팬츠,반바지,무지,포켓,봄 여름,심플베이직,0.0
2,[DAYONE LABEL]슬로 보트넥티,티셔츠,무지,보트넥,봄 가을,청순,0.0
3,[DAYONE LABEL]워프 스퀘어롱원피스,롱원피스,데님,크롭,여름,청순,0.0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),스키니,무지,슬림,겨울,캐주얼,0.0


In [67]:
df=df[['상품명','스타일','계절','카테고리','모양','패턴재질']]

In [68]:
df = pd.get_dummies(data = df, columns = ['스타일'], prefix = '스타일')
df = pd.get_dummies(data = df, columns = ['계절'], prefix = '계절')
df = pd.get_dummies(data = df, columns = ['카테고리'], prefix = '카테고리')
df = pd.get_dummies(data = df, columns = ['모양'], prefix = '모양')
df = pd.get_dummies(data = df, columns = ['패턴재질'], prefix = '패턴재질')
df.head()

,상품명,스타일_데일리,스타일_러블리,스타일_로맨틱,스타일_모던,스타일_모던시크,스타일_미니멀,스타일_볼드,스타일_섹시,스타일_스포티,...,패턴재질_퀼팅,패턴재질_크랙,패턴재질_털,패턴재질_트위드,패턴재질_퍼,패턴재질_펀칭,패턴재질_프린팅,패턴재질_플라워,패턴재질_하찌,패턴재질_헤링본
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,[DAYONE LABEL]로티 숏팬츠,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[DAYONE LABEL]슬로 보트넥티,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,[DAYONE LABEL]워프 스퀘어롱원피스,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [70]:
df.set_index(['상품명'],inplace=True)

In [71]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(df, df)

In [72]:
cosine_sim

array([[5., 1., 1., ..., 0., 0., 1.],
       [1., 5., 1., ..., 1., 0., 1.],
       [1., 1., 5., ..., 0., 1., 1.],
       ...,
       [0., 1., 0., ..., 5., 0., 0.],
       [0., 0., 1., ..., 0., 5., 1.],
       [1., 1., 1., ..., 0., 1., 5.]])

In [73]:
indices =pd.Series(range(1845),index=df.index)
print(indices.head())

상품명
[DAYONE LABEL]렌프 자켓원피스(벨트set)      0
[DAYONE LABEL]로티 숏팬츠               1
[DAYONE LABEL]슬로 보트넥티              2
[DAYONE LABEL]워프 스퀘어롱원피스           3
[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)    4
dtype: int64


In [74]:
indices1=pd.Series(df.index)
print(indices1.head())

0      [DAYONE LABEL]렌프 자켓원피스(벨트set)
1               [DAYONE LABEL]로티 숏팬츠
2              [DAYONE LABEL]슬로 보트넥티
3           [DAYONE LABEL]워프 스퀘어롱원피스
4    [DAYONE LABEL]케이즈 하이스키니진 (히든밴딩)
Name: 상품명, dtype: object


In [75]:
idx = indices['[DAYONE LABEL]렌프 자켓원피스(벨트set)']

In [76]:
sim_scores = list(enumerate(cosine_sim[idx]))

In [77]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

In [78]:
sim_scores = sim_scores[1:11]

In [79]:
movie_indices = [i[0] for i in sim_scores]

In [80]:
movie_indices

[41, 76, 211, 216, 217, 242, 264, 267, 301, 302]

In [81]:
indices1[movie_indices]

41        [La belle rose]엘리제 스퀘어넥 원피스(벨트set)
76                             러바드 플레어 니트원피스
211           [La belle rose]돌체 자켓원피스(벨트set)
216              [La belle rose]원앤 슬림핏 자켓원피스
217    [La belle rose]유리아 새틴 원피스(+4cm기장추가옵션)
242                            마띠 진주 뷔스티에원피스
264                        블린 브이니트원피스(벨트set)
267                                샤르망 진주가디건
301                           [울30%]로나 트위드자켓
302                    [울50%]헤리 더블롱코트(벨트set)
Name: 상품명, dtype: object

In [82]:
dfs=pd.read_excel('아이템기반최종데이터.xlsx')
dfs=dfs[['상품명','카테고리','패턴재질','모양','계절','스타일']]
dfs.head()

,상품명,카테고리,패턴재질,모양,계절,스타일
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),미니원피스,무지,단추,겨울,모던시크
1,[DAYONE LABEL]로티 숏팬츠,반바지,무지,포켓,봄 여름,심플베이직
2,[DAYONE LABEL]슬로 보트넥티,티셔츠,무지,보트넥,봄 가을,청순
3,[DAYONE LABEL]워프 스퀘어롱원피스,롱원피스,데님,크롭,여름,청순
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),스키니,무지,슬림,겨울,캐주얼


In [83]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]
    
    result=pd.DataFrame(indices1[movie_indices])
    laster=pd.merge(result,dfs,how='left' )
    laster=laster.drop_duplicates(['상품명'])
    
    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return laster

In [85]:
get_recommendations('[La belle rose]론다 플리츠 미니스커트')

,상품명,카테고리,패턴재질,모양,계절,스타일
0,민니 핀턱 주름스커트,미니스커트,무지,핀턱,봄 가을,러블리
3,블라디 플리츠스커트,미니스커트,무지,플리츠,가을,러블리
5,아세르 플리츠 미니스커트,미니스커트,무지,플리츠,봄 여름 가을,러블리
8,피버 플리츠 미니스커트,미니스커트,무지,플리츠,봄 여름,러블리
13,슬라 린넨 플리츠스커트,미니스커트,무지,플리츠,여름,러블리
15,케즈 플리츠 롱스커트,롱스커트,무지,플리츠,봄 가을,러블리
16,포레트 랩 미니스커트,미니스커트,무지,랩,봄 가을,러블리
17,[La belle rose]데이니 여리핏 브이트임티,티셔츠,무지,유넥,봄 가을,러블리
21,[La belle rose]딜러스 포켓 뷔스티에원피스(+6cm기장추가옵션),미니원피스,무지,뷔스티에,봄 가을,러블리
24,[La belle rose]머스크 피케이 카라셔츠,셔츠/남방,무지,카라,봄 가을,러블리
